In [ ]:
pip install yt-dlp


: 

In [3]:
import os
import yt_dlp
from moviepy.editor import VideoFileClip
from deepgram import DeepgramClient, PrerecordedOptions, FileSource
from gtts import gTTS
import json

video_url = "https://youtu.be/vk_xq1P7vIU?si=v9-noi_NXARDLVmn"
save_path = r"C:\Users\Nincey Infanta\Documents\yardstick\downloaded/video.mp4"
DG_KEY = "fa16c4732120dd95e1096cf968fb9f3aa8cb8e51"  

audio_path = os.path.join(save_path, "extracted_audio.mp3")
transcript_file = os.path.join(save_path, "transcript.json")
tts_output = os.path.join(save_path, "final_speech.mp3")



def download_youtube_video_simple(url, save_path):
    ydl_opts = {
        'outtmpl': os.path.join(save_path, '%(title)s.%(ext)s'),
        'format': 'best[ext=mp4][vcodec!=none][acodec!=none]',  
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
        print(f"Downloaded video to: {filename}")
        return filename



def extract_audio(video_path, audio_output_path):
    video = VideoFileClip(video_path)
    video.audio.write_audiofile(audio_output_path)
    print(f"Audio extracted to: {audio_output_path}")



def transcribe_audio(audio_file_path, transcript_output, api_key):
    deepgram = DeepgramClient(api_key)

    with open(audio_file_path, "rb") as audio_file:
        buffer_data = audio_file.read()

    payload: FileSource = { "buffer": buffer_data }
    options = PrerecordedOptions(model="nova-2", smart_format=True)

    response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

    with open(transcript_output, "w") as f:
        f.write(response.to_json(indent=4))

    print("Transcription saved to:", transcript_output)


def convert_text_to_speech(transcript_path, output_path):
    with open(transcript_path, "r") as file:
        data = json.load(file)
        text = data['results']['channels'][0]['alternatives'][0]['transcript']

    tts = gTTS(text)
    tts.save(output_path)
    print(f"Text-to-speech saved to: {output_path}")


if __name__ == "__main__":
    os.makedirs(save_path, exist_ok=True)

    video_file_path = download_youtube_video_simple(video_url, save_path)
    extract_audio(video_file_path, audio_path)
    transcribe_audio(audio_path, transcript_file, DG_KEY)
    convert_text_to_speech(transcript_file, tts_output)

    print("\n🎉 All steps completed successfully!")


ModuleNotFoundError: No module named 'yt_dlp'

In [5]:
from moviepy.editor import AudioFileClip
from deepgram import DeepgramClient, PrerecordedOptions, FileSource

def transcribe_audio(audio_file_path, transcript_output, api_key):
    deepgram = DeepgramClient(api_key)

    
    duration = AudioFileClip(audio_file_path).duration

    with open(audio_file_path, "rb") as audio_file:
        buffer_data = audio_file.read()

    payload: FileSource = {"buffer": buffer_data}

    
    options = PrerecordedOptions(
        model="nova-2",
        smart_format=True,
        language="en",
        punctuate=True,
        utterances=False,
        diarize=False,
        multichannel=False,
        tier="enhanced",
        redact={"policy": "none"},
        numerals=True,
        filler_words=False,
        paragraph=True,
        **{"extra": {"duration": duration}}
    )

    response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

    with open(transcript_output, "w", encoding="utf-8") as f:
        f.write(response.to_json(indent=4))

    print("✅ Full transcription saved to:", transcript_output)


In [ ]:
import re

with open(transcript_file, "r") as file:
    data = json.load(file)
    transcript_text = data['results']['channels'][0]['alternatives'][0]['transcript']


sentences = re.split(r'(?<=[.!?])\s+', transcript_text.strip())


with open(os.path.join(save_path, "transcript.txt"), "w", encoding="utf-8") as txt_file:
    for sentence in sentences:
        txt_file.write(sentence.strip() + "\n")

print("Transcript saved with clean line-by-line formatting.")


Transcript saved with clean line-by-line formatting.
